In [3]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

## IO markup

### Train

In [4]:
from modules.data import bert_data

In [5]:
train_df_path = "/home/eartemov/ae/work/factRuEval-2016/dev.csv"
valid_df_path = "/home/eartemov/ae/work/factRuEval-2016/test.csv"

In [6]:
data = bert_data.LearnData.create(
    train_df_path=train_df_path,
    valid_df_path=valid_df_path,
    idx2labels_path="/home/eartemov/ae/work/factRuEval-2016/idx2labels3.txt",
    clear_cache=True
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [7]:
from modules.models.bert_models import BERTCRF

In [8]:
model = BERTCRF.create(len(data.train_ds.idx2label), crf_dropout=0.3)

In [9]:
from modules.train.train import NerLearner

In [10]:
num_epochs = 100

In [9]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/fre-BERTCRF-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [10]:
model.get_n_trainable_params()

298489

In [ ]:
learner.fit(epochs=num_epochs)

### Eval

#### Load model

In [12]:
from modules.data import bert_data

In [13]:
data = bert_data.LearnData.create(
    train_df_path=train_df_path,
    valid_df_path=valid_df_path,
    idx2labels_path="/home/eartemov/ae/work/factRuEval-2016/idx2labels3.txt",
    clear_cache=False
)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at /home/eartemov/.pytorch_pretrained_bert/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729


In [11]:
from modules.models.bert_models import BERTCRF

In [12]:
model = BERTCRF.create(len(data.train_ds.idx2label), crf_dropout=0.3)

In [13]:
from modules.train.train import NerLearner

In [14]:
num_epochs = 100

In [15]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/fre-BERTCRF-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [16]:
model.get_n_trainable_params()

298489

In [17]:
learner.load_model()

### Predict

In [18]:
from modules.data.bert_data import get_data_loader_for_predict

In [19]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [20]:
preds = learner.predict(dl)

In [21]:
from sklearn_crfsuite.metrics import flat_classification_report

In [22]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [23]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [28]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=data.train_ds.idx2label[5:], digits=4)

In [29]:
print(tokens_report)

              precision    recall  f1-score   support

       I_LOC     0.8761    0.7996    0.8361      1557
       I_PER     0.9543    0.9683    0.9612      2112
       I_ORG     0.7944    0.7700    0.7820      3865

   micro avg     0.8572    0.8317    0.8442      7534
   macro avg     0.8750    0.8460    0.8598      7534
weighted avg     0.8561    0.8317    0.8434      7534



In [26]:
from modules.analyze_utils.plot_metrics import get_bert_span_report
from modules.analyze_utils.utils import voting_choicer, tokens2spans

In [27]:
print(get_bert_span_report(dl, preds, fn=voting_choicer))

              precision    recall  f1-score   support

         LOC     0.7639    0.6869    0.7234      1300
           O     0.9709    0.9799    0.9754     48614
         ORG     0.5976    0.5078    0.5490      1863
         PER     0.8359    0.8094    0.8224      1296

    accuracy                         0.9520     53073
   macro avg     0.7921    0.7460    0.7676     53073
weighted avg     0.9494    0.9520    0.9505     53073

